# Ejercicio 9: Uso de la API de Google Gemini

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de Google Gemini de forma básica

In [ ]:
!pip install -q -U google-genai

In [ ]:
!pip install python-dotenv

In [12]:
import os
from google import genai
from google.colab import userdata

# Se extrae la clave del almacenamiento seguro de Colab
api_key = userdata.get('GEMINI_API_KEY')

# Inicializamos el cliente pasando la API Key explícitamente
client = genai.Client(api_key=api_key)
'''
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explica cómo funciona la IA en pocas palabras"
)
'''
print(response.text)

La IA "aprende" de **grandes cantidades de datos** a identificar **patrones y relaciones**. Con ese conocimiento, puede **tomar decisiones, hacer predicciones o generar contenido** de forma "inteligente".


## 2. Retrieval

### 2.1 Cargo el corpus de 20 new Groups



In [7]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame(newsgroups.data[:5000], columns=["documento"])
df = df.rename(columns={"documento": "text"})
df.head()

,text
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...


### 2.2 Transformo a embeddings

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

# Limpieza basica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

print(embeddings.shape, embeddings.dtype)


In [9]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Dallas"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

### 2.3 Creo una query y hago la búsqueda

In [10]:
# Calculamos la similitud (al estar normalizados, el producto punto es igual a la similitud coseno)
# query_vec tiene forma (1, D) y embeddings tiene forma (N, D)
similarities = np.dot(embeddings, query_vec.T).flatten()

# Obtenemos los índices de los 5 resultados con mayor puntuación
top_k = 5
top_indices = np.argsort(similarities)[-top_k:][::-1]

# Creamos un DataFrame con los resultados
results = chunks_df.iloc[top_indices].copy()
results["score"] = similarities[top_indices]

print(f"Top {top_k} resultados para la consulta: '{query_text}'")
results[["score", "text"]]

Top 5 resultados para la consulta: 'Dallas'


,score,text
8343,0.795718,...
307,0.795718,...
7,0.793483,AE is in Dallas...try 214/241-6060 or 214/241-...
4187,0.790328,"This seems appropriate, somehow...>:-)> [....]..."
10125,0.785273,------------------------Tom Hyatt I'm a diehar...


Obtengo los 5 documentos más similares a mi query

In [11]:
# 1. Preparar el contexto uniendo los fragmentos recuperados
context = "\n---\n".join(results["text"].tolist())

# 2. Crear el prompt para Gemini
prompt = f"""
Eres un asistente experto. Utiliza la siguiente información extraída de un corpus de noticias
para responder a la pregunta del usuario de forma concisa.
Si la respuesta no está en el contexto, indica que no tienes suficiente información.

Contexto:
{context}

Pregunta: {query_text}
Respuesta:
"""

# 3. Llamada a la API de Gemini
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=prompt
)

print("\n--- Respuesta de Gemini ---")
print(response.text)


--- Respuesta de Gemini ---
Según el contexto proporcionado, sobre **Dallas** se menciona lo siguiente:

1.  **Ubicación de AE:** La entidad "AE" se encuentra en Dallas, con los números de teléfono 214/241-6060 y 214/241-0055.
2.  **Referencia satírica:** Se menciona humorísticamente que Koresh está en un condominio de tiempo compartido en Dallas junto a Elvis, JFK y J.R. "Bob" Dobbs.
